In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

### Análisis de Polaridad de los tweets

In [2]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

def polaridad_vader(lim_pos=0.25, lim_neg=-0.25, train=True):
    '''
    input = archivo de tweets
    output = dataframe con cantidad de tweets pos, neg y neutros agrupados por fecha
    '''
    
    # armo el nombre del archivo a leer de los parametros
    dir_name = 'data'    
    file_name = 'tweets_train.csv' if train else 'tweets_test.csv'
          
    filename  = os.path.join(dir_name, file_name)
    
    # leo archivo y le cambio los nombres a las columnas
    tweets = pd.read_csv(filename, index_col=0)
    
    # defino la polaridad usando libreria Vader
    sid = SentimentIntensityAnalyzer()
    tweets['polarity'] = [sid.polarity_scores(str(x))['compound'] for x in tweets['text']]
    
    # conteo de resultados en base a los límites predefinidos
    tweets.loc[tweets.polarity >= lim_pos, 'pos'] = 1
    tweets.loc[tweets.polarity <= lim_neg, 'neg'] = 1
    tweets.loc[((tweets.polarity < lim_pos) & (tweets.polarity > lim_neg)), 'neu'] = 1
    
    # fillna con 0
    tweets[['pos', 'neg', 'neu']] = tweets[['pos', 'neg', 'neu']].fillna(value=0)
    
    # selecciono las columnas para trabajar
    tweets = tweets.loc[:, ['date', 'pos', 'neg', 'neu']]
    
    # agrupo tweets por fecha, sumarizando la cantidad de pos, neg y neutros
    grouped = tweets.groupby('date').sum()
    
    # retorno df con resultados
    return grouped
    

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/arturo.torrestey/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
from textblob import TextBlob

def polaridad_textblob(lim_pos=0.25, lim_neg=-0.25, train=True):
    '''
    input = archivo de tweets
    output = dataframe con cantidad de tweets pos, neg y neutros agrupados por fecha
    '''
    
    # armo el nombre del archivo a leer de los parametros
    dir_name = 'data'    
    file_name = 'tweets_train.csv' if train else 'tweets_test.csv'
          
    filename  = os.path.join(dir_name, file_name)
    
    # leo archivo y le cambio los nombres a las columnas
    tweets = pd.read_csv(filename, index_col=0)
    
    # defino la polaridad usando libreria TextBlob
    tweets['polarity'] = [TextBlob(str(x)).sentiment[0] for x in tweets['text']]
    
    # conteo de resultados en base a los límites predefinidos
    tweets.loc[tweets.polarity >= lim_pos, 'pos'] = 1
    tweets.loc[tweets.polarity <= lim_neg, 'neg'] = 1
    tweets.loc[((tweets.polarity < lim_pos) & (tweets.polarity > lim_neg)), 'neu'] = 1
    
    # fillna con 0
    tweets[['pos', 'neg', 'neu']] = tweets[['pos', 'neg', 'neu']].fillna(value=0)
    
    # selecciono las columnas para trabajar
    tweets = tweets.loc[:, ['date', 'pos', 'neg', 'neu']]
    
    # agrupo tweets por fecha, sumarizando la cantidad de pos, neg y neutros
    grouped = tweets.groupby('date').sum()
    
    # retorno df con resultados
    return grouped
    

In [4]:
sent_train = polaridad_vader(train=True)
sent_test  = polaridad_vader(train=False)

### Armado de datasets finales

In [7]:
prices_train = pd.read_csv('data/prices_train.csv', index_col=0)
prices_test = pd.read_csv('data/prices_test.csv', index_col=0)

In [8]:
trainset = pd.merge(sent_train, prices_train, on='date', how='left')
testset = pd.merge(sent_test, prices_test, on='date', how='left')

In [9]:
trainset.to_csv('data/trainset.csv')
testset.to_csv('data/testset.csv')